In [132]:
import cv2
import sys
import numpy as np
import os
import glob
from matplotlib import pyplot as plt

In [133]:
# https://youbidan.tistory.com/19

In [141]:
def make_bbox(img_path, save_path, idx):
    image = cv2.imread(img_path)
    image_gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # 원본 이미지 표시
    #cv2.imshow('image', cv2.resize(image, dsize = (0, 0), fx = 0.5, fy = 0.5))
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

    # grayscale 이미지 표시
    #cv2.imshow('image_gray', cv2.resize(image_gray, dsize = (0, 0), fx = 0.5, fy = 0.5)) 
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

    blur = cv2.GaussianBlur(image_gray, ksize=(3,3), sigmaX=0)
    ret, thresh1 = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY)

    # canny edge 이미지
    edged = cv2.Canny(blur, 10, 250)
    #cv2.imshow('Edged', cv2.resize(edged, dsize = (0, 0), fx = 0.5, fy = 0.5))
    #cv2.waitKey(0)

    # closed 이미지
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
    closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)
    #cv2.imshow('closed', cv2.resize(closed, dsize = (0, 0), fx = 0.5, fy = 0.5))
    #cv2.waitKey(0)

    # contour 생성
    contours, _ = cv2.findContours(closed.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    total = 0

    # 이미지에 컨투어 표시
    #contours_image = cv2.drawContours(image, contours, -1, (0,255,0), 3)
    #cv2.imshow('contours_image', cv2.resize(contours_image, dsize = (0, 0), fx = 0.5, fy = 0.5))
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

    contours_xy = np.array(contours)
    #print(contours_xy)
    contours_xy.shape

    # x의 min과 max 찾기
    x_min, x_max = 0, 0
    value = list()
    # for i in range(len(contours)) 형태로 변경
    for i in range(len(contours_xy)):
        for j in range(len(contours_xy[i])):
            value.append(contours_xy[i][j][0][0]) #네번째 괄호가 0일때 x의 값
            x_min = min(value)
            x_max = max(value)
    #print(x_min)
    #print(x_max)
 
    # y의 min과 max 찾기
    y_min, y_max = 0, 0
    value = list()
    for i in range(len(contours_xy)):
        for j in range(len(contours_xy[i])):
            value.append(contours_xy[i][j][0][1]) #네번째 괄호가 0일때 x의 값
            y_min = min(value)
            y_max = max(value)
    #print(y_min)
    #print(y_max)

    # 이미지에 bbox 표시
    boxed_image = cv2.rectangle(image.copy(), (x_min, y_min), (x_max, y_max), (0,255,0), 3)
    #cv2.imshow('boxed_image', cv2.resize(boxed_image, dsize = (0, 0), fx = 0.5, fy = 0.5))
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

    # image trim 하기
    x = x_min
    y = y_min
    w = x_max-x_min
    h = y_max-y_min

    # crop한 이미지 저장
    croped_img = image[y:y+h, x:x+w]
    file_name = img_path.split('\\')[-1]
    cv2.imwrite(f'{save_path}/{file_name}', croped_img)
    #org_image = cv2.imread('croped_img.jpg')

    #cv2.imshow('org_image', cv2.resize(org_image, dsize = (0, 0), fx = 0.5, fy = 0.5))
    #cv2.waitKey(0)
    cv2.destroyAllWindows()

    # yolov5 형식으로 txt 파일 저장
    yolo_x = ((x_min + x_max)*0.5)/image.shape[1]
    yolo_y = ((y_min + y_max)*0.5)/image.shape[0]
    yolo_w = ((x_max - x_min))/image.shape[1]
    yolo_h = ((y_max - y_min))/image.shape[0]

    # 이미지의 클래스 추가해야함
    with open(f'{save_path}/{file_name[:-4]}.txt', 'w') as file:
        file.write(f'{yolo_x} {yolo_y} {yolo_w} {yolo_h}')

In [136]:
def check_bbox(img_path, txt_path):
    img = cv2.imread(img_path)
    file_name = img_path.split('\\')[-1]
    with open(f'{txt_path}/{file_name[:-4]}.txt') as file:
        label_list = file.read().split(' ')

    img_w = float(img.shape[1])
    img_h = float(img.shape[0])

    x = img_w*(float(label_list[0]) - 0.5*float(label_list[2]))
    y = img_h*(float(label_list[1]) - 0.5*float(label_list[3]))
    w = img_w*float(label_list[2])
    h = img_h*float(label_list[3])
    
    cv2.rectangle(img, (int(x), int(y), int(w), int(h)), (0, 255, 0), 2)
    cv2.imshow('rect', img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [137]:
index = 24
img_list = glob.glob('../dataset/detection_sample/*.jpg')

In [138]:
make_bbox(img_list[index], 'contour_result', 0)

In [139]:
check_bbox(img_list[index], 'contour_result')